In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [9]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    val controlChannel = Channel<Int>()
    launch { // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) { delay(400); channel.send(2 * x); }
        controlChannel.send(0) // 감독관한테 나 일 다했어
        println("sending even number done")
    }
    launch { // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) { delay(800); channel.send(2 * x - 1) }
        channel.close()
        controlChannel.send(1) // 감독관한테 나 일 다했어
        println("sending odd number done")
    }
    launch { // 감독관 작업?
        controlChannel.receive()
        controlChannel.receive()
        channel.close()
        controlChannel.close()
    }
    // here we print five received integers:
    for(x in 1..10) { println(channel.receive()) }
    println("Done!")
}

2
1
4
6
3
8
sending even number done
10
5
7
sending odd number done
9
Done!


In [6]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) { delay(400); channel.send(x) }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) { println(x) }
    // for(x in 1..5) { println(channel.receive()) }
    println("Done!")
}

1
2
3
4
5
6
7
8
9
10
Done!
